In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import os
import glob
import pandas as pd

tiempo_inicio_script = time.time() - 2

# ==========================================\n",
# 1. CONFIGURACIÓN DEL NAVEGADOR Y RUTAS\n",
# ==========================================\n",
opciones = webdriver.ChromeOptions()

# Definimos las rutas de trabajo
raw_dir = r"C:\Users\Edward\Desktop\Bancomext\Estatales\data\raw"
intermediate_dir = r"C:\Users\Edward\Desktop\Bancomext\Estatales\data\intermediate"

# Aseguramos que la carpeta intermediate exista
os.makedirs(intermediate_dir, exist_ok=True)

prefs = {"download.default_directory" : raw_dir}
opciones.add_experimental_option("prefs", prefs)

# Forzamos el idioma a español para evitar que Tableau cambie "(Todo)" por "(All)"
opciones.add_argument("--lang=es-MX") 

driver = webdriver.Chrome(options=opciones)
driver.get("https://public.tableau.com/app/profile/imss.cpe/viz/Histrico_4/Empleo_h?publish=yes")

wait = WebDriverWait(driver, 20)

# Inicializamos el DataFrame maestro vacío en memoria
df_master = pd.DataFrame()

# ==========================================
# 2. PREPARACIÓN DEL DASHBOARD
# ==========================================
# A. Manejar el banner de cookies
try:
    print("Buscando banner de cookies...")
    btn_cookies = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
    btn_cookies.click()
    print("✅ Cookies aceptadas.")
    time.sleep(1) 
except TimeoutException:
    print("ℹ️ No apareció el aviso de cookies.")

# B. Entrar al iFrame
print("Buscando el iFrame del dashboard...")
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

# C. Ir a la pestaña "Cifras de salario"
print("Navegando a 'Cifras de salario'...")
xpath_pestana = "//div[contains(@class, 'tabStoryPointContent') and contains(normalize-space(), 'Cifras de salario')]"
tab_salario = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_pestana)))
tab_salario.click()
time.sleep(2) # Espera larga para que cargue toda la base del IMSS

# D. Abrir el filtro de Entidad por primera vez
print("Abriendo menú de Entidad...")
xpath_filtro_entidad = "(//span[@role='combobox'])[1]"
filtro_entidad = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_filtro_entidad)))
filtro_entidad.click()
time.sleep(2)

# E. Deseleccionar "(Todo)"
print("Deseleccionando la casilla principal...")
xpath_todo_box = "//div[@role='checkbox' and .//a[@title='(Todo)' or @title='(All)']]//div[@class='fakeCheckBox']"
try:
    box_todo = wait.until(EC.presence_of_element_located((By.XPATH, xpath_todo_box)))
    ActionChains(driver).move_to_element(box_todo).click().perform()
    print("✅ Casilla principal deseleccionada.")
except Exception as e:
    print(f"⚠️ Error al quitar (Todo): {e}")
time.sleep(2) 


# ==========================================
# 3. EL BUCLE DE EXTRACCIÓN (Optimizado con Buscador)
# ==========================================
estados = [
    "Aguascalientes", "Baja California", "Baja California Sur", "Campeche", "Chiapas", 
    "Chihuahua", "Ciudad de México", "Coahuila de Zaragoza", "Colima", "Durango", 
    "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "México", "Michoacán de Ocampo", 
    "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", "Querétaro", 
    "Quintana Roo", "San Luis Potosí", "Sinaloa", "Sonora", "Tabasco", 
    "Tamaulipas", "Tlaxcala", "Veracruz de Ignacio de la Llave", "Yucatán", "Zacatecas"
]

xpath_search = "//textarea[contains(@class, 'QueryBox')]"

for estado in estados:
    print(f"\n--- Iniciando extracción de: {estado} ---")
    try:
        # --- A. Buscar y Seleccionar Estado ---
        print("Filtrando estado en la barra de búsqueda...")
        search_box = wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
        
        # Limpiamos la caja de texto y escribimos el estado
        search_box.send_keys(Keys.CONTROL, "a")
        search_box.send_keys(Keys.DELETE)
        search_box.send_keys(estado)
        time.sleep(1) # Pausa VITAL para que Tableau oculte los demás estados
        
        xpath_estado_box = f"//div[@role='checkbox' and .//a[@title='{estado}']]//div[@class='fakeCheckBox']"
        box_estado = wait.until(EC.presence_of_element_located((By.XPATH, xpath_estado_box)))
        
        # Ya no hay scroll. Hacemos clic directo porque siempre estará visible
        ActionChains(driver).move_to_element(box_estado).pause(0.5).click().perform()
        
        # Cerrar el menú
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        
        print(f"Esperando a que carguen los datos de {estado}...")
        time.sleep(2) 
        
        # --- B. Proceso de Descarga ---
        print("Iniciando secuencia de descarga...")
        xpath_btn_descarga = "//button[@data-tb-test-id='viz-viewer-toolbar-button-download']"
        btn_descarga = wait.until(EC.presence_of_element_located((By.XPATH, xpath_btn_descarga)))
        driver.execute_script("arguments[0].click();", btn_descarga)
        time.sleep(1) 

        xpath_crosstab = "//div[@data-tb-test-id='download-flyout-download-crosstab-MenuItem']"
        btn_crosstab = wait.until(EC.presence_of_element_located((By.XPATH, xpath_crosstab)))
        driver.execute_script("arguments[0].click();", btn_crosstab)
        
        print("Seleccionando formato CSV...")
        xpath_csv_label = "//label[@data-tb-test-id='crosstab-options-dialog-radio-csv-Label']"
        btn_csv = wait.until(EC.presence_of_element_located((By.XPATH, xpath_csv_label)))
        driver.execute_script("arguments[0].click();", btn_csv)
        time.sleep(1) 

        xpath_descarga_final = "//button[@data-tb-test-id='export-crosstab-export-Button']"
        btn_descarga_final = wait.until(EC.presence_of_element_located((By.XPATH, xpath_descarga_final)))
        driver.execute_script("arguments[0].click();", btn_descarga_final)

        print(f"📥 Descargando archivo de {estado}...")
        time.sleep(3) 
        print(f"✅ Descarga completada.")
        
        # --- NUEVO: Procesamiento, integración y eliminación ---
        print(f"🔄 Procesando datos de {estado}...")
        
        # 1. Búsqueda quirúrgica del archivo descargado
        prefijo_descarga = "Salario" # Tableau nombra el archivo empezando con "Salario"
        
        patron_busqueda = os.path.join(raw_dir, f"{prefijo_descarga}*.csv")
        archivos_candidatos = glob.glob(patron_busqueda)
        
        # Filtrar SOLO los creados/modificados DESPUÉS de que inició el script
        archivos_validos = [f for f in archivos_candidatos if os.path.getmtime(f) >= tiempo_inicio_script]
        
        if archivos_validos:
            archivo_reciente = max(archivos_validos, key=os.path.getmtime)
            
            # 2. Leer el archivo saltando la fila 1. 
            df_temp = pd.read_csv(
                archivo_reciente, 
                skiprows=1, 
                header=None, 
                usecols=[0, 1], 
                names=['Fecha', estado],
                encoding='utf-16',
                sep='\t'
            )
            
            # --- NUEVO: Limpieza de la columna Fecha ("abril de 1998" -> "abril 1998") ---
            # Reemplazamos " de " por un espacio y quitamos espacios en los extremos
            df_temp['Fecha'] = df_temp['Fecha'].astype(str).str.replace(' de ', ' ', regex=False).str.strip()
            
            # 3. Integrar al DataFrame maestro usando un outer join
            if df_master.empty:
                df_master = df_temp
            else:
                df_master = pd.merge(df_master, df_temp, on='Fecha', how='outer')
                
            # 4. Eliminar el archivo individual descargado
            try:
                os.remove(archivo_reciente)
                print(f"🗑️ Archivo temporal de {estado} eliminado.")
            except Exception as e:
                print(f"⚠️ No se pudo eliminar el archivo temporal: {e}")
        else:
            print(f"⚠️ No se detectó ningún archivo CSV descargado para {estado}.")

        # --- C. Reset para el siguiente ciclo ---
        print("Regresando la vista hacia arriba...")
        xpath_filtro_entidad = "(//span[@role='combobox'])[1]"
        filtro_entidad = wait.until(EC.presence_of_element_located((By.XPATH, xpath_filtro_entidad)))
        
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", filtro_entidad)
        time.sleep(1) 
        
        print("Reabriendo filtro de Entidad...")
        ActionChains(driver).move_to_element(filtro_entidad).pause(0.5).click().perform()
        time.sleep(1) 
        
        # --- LA BUENA PRÁCTICA QUE SUGERISTE ---
        print(f"Asegurando búsqueda de {estado} para deselección...")
        search_box = wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
        search_box.send_keys(Keys.CONTROL, "a")
        search_box.send_keys(Keys.DELETE)
        search_box.send_keys(estado)
        time.sleep(1) # Pausa para que el filtro se aplique
        
        print(f"Limpiando selección de {estado}...")
        box_estado_limpiar = wait.until(EC.presence_of_element_located((By.XPATH, xpath_estado_box)))
        ActionChains(driver).move_to_element(box_estado_limpiar).pause(0.5).click().perform()
        time.sleep(1) 
        
        # OBLIGATORIO: Dejar la caja de búsqueda en blanco para el próximo estado
        search_box.send_keys(Keys.CONTROL, "a")
        search_box.send_keys(Keys.DELETE)
        time.sleep(1)
        
    except Exception as e:
        print(f"❌ Error procesando {estado}: {e}")

print("¡Proceso Finalizado!")

# ==========================================
# 3.5 ORDENAR DATOS CRONOLÓGICAMENTE
# ==========================================
if not df_master.empty:
    print("\n📅 Ordenando las fechas cronológicamente...")
    
    # Diccionario para mapear los meses en español a números
    meses = {
        'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
        'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
        'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
    }
    
    # Creamos una columna temporal en minúsculas para trabajar
    df_master['Fecha_Temp'] = df_master['Fecha'].str.lower().str.strip()
    
    # Reemplazamos el nombre del mes por su número con una diagonal (ej. "abril 1998" -> "04/ 1998")
    for mes_nombre, mes_numero in meses.items():
        df_master['Fecha_Temp'] = df_master['Fecha_Temp'].str.replace(mes_nombre, f"{mes_numero}/", regex=False)
        
    # Quitamos los espacios restantes para que quede un formato limpio "04/1998"
    df_master['Fecha_Temp'] = df_master['Fecha_Temp'].str.replace(' ', '', regex=False)
    
    # Convertimos esa columna texto a un formato Datetime real de Pandas
    df_master['Fecha_Temp'] = pd.to_datetime(df_master['Fecha_Temp'], format='%m/%Y', errors='coerce')
    
    # Ordenamos de la fecha más antigua a la más reciente y reseteamos el índice
    df_master = df_master.sort_values(by='Fecha_Temp', ascending=True).reset_index(drop=True)
    
    # Eliminamos la columna temporal, ya cumplió su propósito
    df_master = df_master.drop(columns=['Fecha_Temp'])

# ==========================================\n",
# 4. GUARDADO DEL ARCHIVO MAESTRO\n",
# ==========================================\n",
if not df_master.empty:
    print("\n💾 Guardando el consolidado nacional en la carpeta intermediate...")
    ruta_salida = os.path.join(intermediate_dir, "salarios_imss.csv")
    
    # Guardamos sin el índice y con formato utf-8-sig para no tener problemas con acentos en Excel
    df_master.to_csv(ruta_salida, index=False, encoding='utf-8-sig')
    print(f"✅ ¡Éxito! Archivo maestro guardado en:\n{ruta_salida}")
else:
    print("\n⚠️ El DataFrame maestro está vacío. No se guardó ningún archivo.")

# --- NUEVO: Cerrar el navegador y finalizar la sesión ---
print("\nCerrando el navegador...")
driver.quit()

Buscando banner de cookies...
✅ Cookies aceptadas.
Buscando el iFrame del dashboard...
Navegando a 'Cifras de salario'...
Abriendo menú de Entidad...
Deseleccionando la casilla principal...
✅ Casilla principal deseleccionada.

--- Iniciando extracción de: Aguascalientes ---
Filtrando estado en la barra de búsqueda...
Esperando a que carguen los datos de Aguascalientes...
Iniciando secuencia de descarga...
Seleccionando formato CSV...
📥 Descargando archivo de Aguascalientes...
✅ Descarga completada.
🔄 Procesando datos de Aguascalientes...
🗑️ Archivo temporal de Aguascalientes eliminado.
Regresando la vista hacia arriba...
Reabriendo filtro de Entidad...
Asegurando búsqueda de Aguascalientes para deselección...
Limpiando selección de Aguascalientes...

--- Iniciando extracción de: Baja California ---
Filtrando estado en la barra de búsqueda...
Esperando a que carguen los datos de Baja California...
Iniciando secuencia de descarga...
Seleccionando formato CSV...
📥 Descargando archivo de B